In [ ]:
from collections import defaultdict, deque
import sys

sys.setrecursionlimit(10000)

def create_empty_tree(n):
    if n <= 0:
        return []
    return [[] for _ in range(n)]

def add_undirected_edge(tree, from_node, to_node):
    if from_node >= len(tree)  or  to_node >= len(tree):
        raise ValueError("Invalid node index.")
    tree[from_node].append(to_node)
    tree[to_node].append(from_node)

def find_tree_centers(tree):
    n = len(tree)
    if n == 0:
        return []
    
    degrees = [len(neighbors) for neighbors in tree]
    leaves = [i for i in range(n) if degrees[i] == 1]
    processed_leaves = len(leaves)

    while processed_leaves < n:
        new_leaves = []
        for leaf in leaves:
            for neighbor in tree[leaf]:
                degrees[neighbor] -= 1
                if degrees[neighbor] == 1:
                    new_leaves.append(neighbor)
        processed_leaves += len(new_leaves)
        leaves = new_leaves
    return leaves


def bfs_encode_tree(tree):
    if not tree:
        return ""
    if len(tree) == 1:
        return "()"
    
    root = find_tree_centers(tree)[0] # case we have 2 centers select [0] for generalization
    visited = [False] * len(tree)
    labels  = ["()"]  * len(tree)

    q = deque([root])
    visited[root] = True
    while q:
        at = q.popleft()
        children = []
        for neighbor in tree[at]:
            if not visited[neighbor]:
                visited[neighbor] = True
                q.append(neighbor)
            elif labels[neighbor] != "()":  # Already processed child
                # will enter this case when get the leafes so will start backtarck
                children.append(labels[neighbor])
        
        children.sort()
        labels[at] = f"({''.join(children)})"
    return labels[root]

def ahu_encode_tree(tree):
    def dfs(node, parent):
        labels = []
        for neighbor in tree[node]:
            if neighbor != parent:
                labels.append(dfs(neighbor, node))
        labels.sort()
        return f"({''.join(labels)})"
    centers = find_tree_centers(tree)            
    if len(centers) == 1:
        return dfs(centers[0], -1)
    elif len(centers) == 2:
        center1, center2 = centers
        return min(dfs(center1, center2), dfs(center2, center1))

def trees_are_isomorphic(tree1, tree2, method="bfs"):
    if method == "bfs":
        return bfs_encode_tree(tree1) == bfs_encode_tree(tree2)
    elif method == "ahu":
        return ahu_encode_tree(tree1) == ahu_encode_tree(tree2)
    else:
        raise ValueError("Unknown method: choose 'bfs' or 'ahu'")


In [ ]:
import math 

class TreeNode:
    def __init__(self, index, parent=None):
        self.index = index 
        self.parent = parent
        self.children = []
        self.n = 0 

    def add_children(self, *nodes):
        for node in nodes:
            self.children.append(node)
        
    def set_size(self, n):
        self.n = n
    
    def size(self):
        return self.n

    def __str__(self):
        return str(self.index)

def build_tree(graph, node, parent = None):
    # construct the tree and return root node
    subtree_node_count = 1
    for neighbor in graph[node.index]:
        if parent is not None and neighbor == parent.index:
            continue
        child = TreeNode(neighbor, node)
        node.add_children(child)
        subtree_node_count += build_tree(graph, child, node).size()
    node.set_size(subtree_node_count)
    return node

def create_tree_from_graph(graph, root_id):
    root = TreeNode(root_id)
    return build_tree(graph, root)

def create_empty_graph(n):
    return [[] for _ in range(n)]

def add_undirected_edge(graph, from_node, to_node):
    # Add an undirected edge between two nodes in the graph
    graph[from_node].append(to_node)
    graph[to_node].append(from_node)


class SparseTable:
    def __init__(self, values): # depth
        self.n = len(values) # lenth of depth array

        self.P = math.floor(math.log2(self.n))
            # - `dp`: A table that stores the minimum values for subarrays of lengths 2^p.
            # - `it`: A table that stores the indices of the minimum values in the `dp` table.
            # - `log2`: Precomputed logarithm values for efficient query processing.
        self.dp = [[0] * self.n for _ in range(self.P + 1)]
        self.it = [[0] * self.n for _ in range(self.P + 1)]
        self.log2 = [0] * (self.n + 1)

        # init from root to any node whthere for dp metric or it metric
        for i in range(self.n):
            self.dp[0][i] = values[i]
            self.it[0][i] = i
        
        for i in range(2, self.n + 1):
            self.log2[i] = self.log2[i // 2] + 1

        for p in range (1, self.p + 1): # level 0 --> done
            for i in range(self.n - (1 << p) + 1):
                        
    def query_index(self, l, r):
        pass

class LowestCommonAncestorEulerTour:
    def __init__(self, root):
        self.n = root.size()
        self.tour_index = 0
        self.node_depth = []
        self.node_order = []
        self.last = [-1] * self.n
        self.sparse_table = None
        self._setup(root)

    def _setup(self, root):
        euler_tour_size = 2 * self.n - 1
        self.node_order = [None] * euler_tour_size
        self.node_depth = [0]    * euler_tour_size 

        self._dfs(root, 0)

        self.sparse_table = SparseTable(self.node_depth)
    

    def _dfs(self, node, depth):
        self._visit(node, depth)
        for child in node.children:
            self._dfs(child, depth + 1)
            self._visit(node, depth)

    def _visit(self, node, depth):
        #Recored is the first pratical operation when start building
        self.node_order[self.tour_index] = node
        self.node_depth[self.tour_index] = depth
        self.last[node.index] = self.tour_index
        self.tour_index += 1
    
    def lca(self, index1, index2):
        l = min(self.last[index1], self.last[index2])
        r = max(self.last[index1], self.last[index2])

        i = self.sparse_table.query_index(l, r)
        return self.node_order[i]


In [9]:
a0 = (1 << 0) 
a1 = (1 << 1) 
a2 = (1 << 2) 
a3 = (1 << 3) 

a0, a1, a2 , a3

(1, 2, 4, 8)

In [1]:
dp = [[0] * 7 for _ in range(3)]
dp

[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]